In [ ]:
#using the helper funtions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
#using the helper functions
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys,walk_through_dir

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [ ]:
unzip_data("10_food_classes_10_percent.zip")

In [ ]:
#checkingout how many imags and subdirectories
walk_through_dir("10_food_classes_10_percent")

In [ ]:
# Create training and test directory paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [ ]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE =32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size = BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                 image_size = IMG_SIZE,
                                                                 label_mode = "categorical",
                                                                 batch_size = BATCH_SIZE)

In [ ]:
train_data_10_percent

# Dataset shapes and visualization or conversion to numpy

In [ ]:
batch_shapes = [spec.shape for spec in train_data_10_percent.element_spec]
print(batch_shapes)


In [ ]:
for image_batch, label_batch in train_data_10_percent.take(1):
    print("Actual Image Batch Shape:", image_batch.shape)  # Should be (32, 224, 224, 3) if batch_size=32
    print("Actual Label Batch Shape:", label_batch.shape)  # Should be (32, 10)


In [ ]:
import tensorflow as tf
import numpy as np

# Assume you have a dataset
dataset = ...  # Your PrefetchDataset or BatchDataset

# Get element_spec
input_spec, label_spec = train_data_10_percent.element_spec

# Generate random tensors with the correct shape (replace None with a fixed batch size)


input_data = tf.random.uniform(shape=(BATCH_SIZE, *input_spec.shape[1:]), dtype=input_spec.dtype)
label_data = tf.random.uniform(shape=(BATCH_SIZE, *label_spec.shape[1:]), dtype=label_spec.dtype)

print("Generated Input Data Shape:", input_data.shape)
print("Generated Label Data Shape:", label_data.shape)


In [ ]:
new_dataset = tf.data.Dataset.from_tensor_slices((input_data, label_data)).batch(BATCH_SIZE)

# Check the shapes of the first batch
for batch in new_dataset.take(1):
    print([tensor.shape for tensor in batch])


In [ ]:
input_np = input_data.numpy()
label_np = label_data.numpy()
print("NumPy Input Shape:", input_np.shape)
print("NumPy Label Shape:", label_np.shape)


# Continuing from there.

In [ ]:
train_data_10_percent.class_names

In [ ]:
import numpy as np

# Get one batch
image_batch, label_batch = next(iter(train_data_10_percent))

# Check min and max pixel values
print("Min pixel value:", np.min(image_batch.numpy()))
print("Max pixel value:", np.max(image_batch.numpy()))


In [ ]:
#see an example of a batch of data
for images,labels in train_data_10_percent.take(1):
  print(images,labels)

#model 0 : building a transfer learning model using the keras Funcitonal API

Functional API provides more flexibility in building models.

In [ ]:
#1. create base model with tf.keras.applications.

base_model = tf.keras.applications.EfficientNetB0(include_top = False)

#2. Freeze the base model( so the underlying pre-trained patterns aren't updated)
base_model.trainable = False

#3. create inputs into our model
inputs = tf.keras.layers.Input(shape = (224,224,3), name = "input_layer")

#4. If using a model like ResNet50V2 we need to normalize inputs (efficient don't need as it has already builtin if using from tf.keras)
#x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

#6. Average pool the ouputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name= "global_average_pooling_layer")(x)
print(f"Shape after Global Average Pooling: {x.shape}")

#7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation = "softmax", name = "output_layer")(x)
print(f"Shape after output layer: {outputs.shape}")

#8. combine the inputs with the outputs into a model.
model_0 = tf.keras.Model(inputs, outputs)

#9. compile the model
model_0.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

#10. fit the model and sae its history
history_10_percent = model_0.fit(train_data_10_percent,
                        epochs = 5,
                        steps_per_epoch = len(train_data_10_percent),
                        validation_data = test_data,
                                 validation_steps = int(0.25 * len(test_data)),
                                 callbacks = [create_tensorboard_callback(dir_name = "transfer_learning",
                                                                           experiment_name = "efficientnetb0_10_percent_data")]
                                 )


In [ ]:
#evaluate on the the full test dataset
model_0.evaluate(test_data)

In [ ]:
#check the layers in our base model.
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number,layer.name)

In [ ]:
base_model.summary()


In [ ]:
model_0.summary()

In [ ]:
#plot loss curves
plot_loss_curves(history_10_percent)

## Getting a feature vector from a trained model

demonstrating the global average pooling 2D layer...

we have a tensor after our model goes through 'base_model' of shape(None, 7,7,1280)

But when it passes through the GlobalAveragePooling2D, it turns into (None, 1280)

try (1,4,4,3)

In [ ]:
# Define the input shape
input_shape = (1,4,4,3)

# Create a random tensor
tf.random.set_seed(42)
input_tensor = tf.random.normal(input_shape)
print(f"Random input tensor : {input_tensor}")
input_tensor.shape



In [ ]:
#pass the ranodm tensor through a global average pooling 2D layer
global_average_pooled_tensor = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
print(f"2D Global Average Pooled random tensor: {global_average_pooled_tensor}")

In [ ]:
global_average_pooled_tensor.shape

In [ ]:
# Let's replicate the GlobalAveragePool2D layer
tf.reduce_mean(input_tensor, axis = [1,2])

# Running a series of transfer learning experiments

1. model_1 : use feature extraction transfer learning with 1% training data wth data augmentation
2. model_2 : use feature extraction transfer learning with 10% training datawith data augmentation
3. model_3 : use fine-tuning transfer learning on 10% of the training data with data augmentation
4. model_4 : use fine-tuning transfer learning on 100% of the training data with data augmentation.

#Getting and preprocessing data for model_1

In [ ]:
#downloading and unziping the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

#unzip the data using the helper_function.py module
unzip_data("10_food_classes_1_percent.zip")

In [ ]:
#Create training and test dirs
train_dir_1_percent = "10_food_classes_1_percent/train"
test_dir_1_percent = "10_food_classes_1_percent/test"

In [ ]:
#How many images are we working with
walk_through_dir("10_food_classes_1_percent")

In [ ]:
#setup data loaders
IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_1_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir_1_percent,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size = BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir_1_percent,
                                                                 image_size = IMG_SIZE,
                                                                 label_mode = "categorical",
                                                                 batch_size = BATCH_SIZE)

#Adding data augmentation into the model.

To add data augmentation to our model, we can use the layers inside:

* tf.keras.layers.experimental.preprocessing()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Create data augmentation stage with horizontal flipping, rotations, zooms, etc
data_augmentation = keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"), # Corrected typo: "horizonatal" to "horizontal"
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    # tf.keras.layers.Rescaling(1./255) #Keep this for models like ResNet50V2 but EfficientNet already has a built-in preprocessing.
], name="data_augmentation")

In [ ]:
#use above or this code instead
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomHeight, RandomWidth
# Create data augmentation stage with horizontal flipping, rotations, zooms, etc
data_augmentation = keras.Sequential([
    RandomFlip("horizontal"), # Corrected typo: "horizonatal" to "horizontal"
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomHeight(0.2),
    RandomWidth(0.2),
    # Rescaling(1./255) #Keep this for models like ResNet50V2 but EfficientNet already has a built-in preprocessing.
], name="data_augmentation")

### Visualize our data augmentation layer

In [ ]:
#view a random image an compare it to ts augmented version
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random

In [ ]:
target_class = random.choice(train_data_1_percent.class_names)